<a href="https://colab.research.google.com/github/Alangonc/healthBot/blob/main/healthBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot Médico Inteligente com Google Gemini

---
**Projeto:** Chatbot de Análise de Receitas Médicas<br>

**Autor:** Alan Gonçalves<br>

**Curso/Disciplina:** Imersão IA 3a. Edição Alura + Google<br>

**Orientador(es):** <br>Fabricio Carraro - Program Manager<br>
Luciano Martins - Developer Advocate, Google IA. <br>
Valquíria Alencar - Instrutora de Data Science

**Data de Início:** 12/05/2025 <br>
**Data da Versão Atual:** 16/05/2025 <br>
**Versão do Código:** v1.0.1-alpha <br>

**Contato:**
*   Email: alangonc@gmail.com
*   LinkedIn: [Seu Perfil LinkedIn](https://www.linkedin.com/in/alangoncalves/)
*   GitHub: [Seu Perfil GitHub](https://github.com/Alangonc)

**Repositório do Projeto:** [HealthBot](https://github.com/Alangonc/healthChatbot)
---

## Visão Geral do Projeto
---
**Chatbot Médico Inteligente com Google Gemini: Análise de Receitas e Estimativa de Custos**

**Visão Geral do Projeto:**

Este projeto demonstra a criação de um chatbot interativo utilizando a API Google Gemini para auxiliar usuários a entenderem suas receitas médicas. O chatbot é capaz de:

1.  **Processar Documentos PDF:** Aceitar o upload de uma receita médica em formato PDF.
2.  **Extrair Informações Relevantes:** Utilizar o poder do modelo multimodal do Gemini para analisar o PDF e extrair uma lista de medicamentos, insumos e equipamentos prescritos, juntamente com suas apresentações e quantidades.
3.  **Estimativa de Custos (Simulada):** Com base nos itens extraídos, o chatbot tenta fornecer uma estimativa de custo consultando (de forma simulada neste protótipo) preços em três grandes redes de farmácias brasileiras (Drogasil, Droga Raia, Drogaria São Paulo).
4.  **Cálculos Auxiliares (Diabetes):** Oferecer assistência com cálculos relacionados ao tratamento de diabetes, como a dose de insulina baseada na contagem de carboidratos (se os parâmetros forem fornecidos pelo usuário ou extraídos da receita).
5.  **Interação Segura:** Fornecer avisos importantes sobre a natureza auxiliar das informações, enfatizando que não substitui o aconselhamento médico profissional.

**Tecnologias Utilizadas:**

*   **Python:** Linguagem de programação principal.
*   **Google Gemini API:** Para as capacidades de IA generativa e análise de documentos.
*   **Google Colab:** Ambiente de desenvolvimento e execução do notebook.
*   **Bibliotecas Python:** `google-generativeai`, `requests`, `beautifulsoup4`, `lxml` (as últimas três para a implementação *real* de web scraping, aqui usadas conceitualmente).

**Objetivo:**

O objetivo é criar uma ferramenta que possa facilitar o entendimento de prescrições médicas complexas e oferecer uma ideia inicial de custos, promovendo maior autonomia e informação ao paciente, sempre com foco na segurança e na responsabilidade.

**Importante:** A funcionalidade de busca de preços neste notebook é **SIMULADA**. Para uma aplicação real, seria necessário implementar web scraping robusto ou integrar APIs de farmácias.

## 01. Install Google Gemini SDK

In [ ]:
# CELL 1: Install Google Gemini SDK and necessary libraries for potential web scraping

# google-genai é um pacote que pode incluir a biblioteca principal ou ser uma dependência.
# google-generativeai é a biblioteca principal e mais recente para interagir com a API Gemini.
# As outras bibliotecas (requests, beautifulsoup4, lxml) seriam para a implementação
# real de web scraping, que aqui é simulada.
!pip install google-generativeai requests beautifulsoup4 lxml
!pip install google-generativeai

## 02. Importing Libraries

In [ ]:
# CELL 2: Importing Libraries

import os
import json # Para parsear (interpretar) strings JSON retornadas pelo LLM
import re   # Para usar expressões regulares (buscar padrões em texto, como números)
import time # Para adicionar pausas (útil em web scraping real para não sobrecarregar servidores)
import math # Para funções matemáticas como math.ceil (arredondar para cima)

from google.colab import userdata # Para acessar "Secrets" no Colab (API Keys)
from google.colab import files    # Para permitir o upload de arquivos do computador do usuário para o Colab
import google.generativeai as genai # Biblioteca principal da API Gemini
from google.api_core import exceptions # Para tratar erros específicos da API do Google

# Bibliotecas para Web Scraping (usadas conceitualmente aqui, mas importantes para a versão real)
import requests # Para fazer requisições HTTP (buscar páginas da web)
from bs4 import BeautifulSoup # Para parsear HTML e extrair dados de páginas web

In [ ]:
# CELL 3: Para verificar a versão instalada

print(f"Versão da biblioteca google-generativeai: {genai.__version__}")

Versão da biblioteca google-generativeai: 0.8.5


## 03. Importing API KEY Gemini

In [ ]:
# CELL 4: Importing API KEY Gemini and Configuring the SDK

# Obtém a GOOGLE_API_KEY armazenada nos "Secrets" do Google Colab.
# É uma prática de segurança não colocar chaves de API diretamente no código.
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# A linha abaixo é opcional se genai.configure() for usado, mas pode ser útil em alguns contextos.
# os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

# Configura a biblioteca Gemini com a chave de API.
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave API configurada com sucesso!")
else:
    # Alerta o usuário se a chave não for encontrada. O chatbot não funcionará sem ela.
    print("ERRO: GOOGLE_API_KEY não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua GOOGLE_API_KEY aos Secrets do Colab para continuar.")
    # Em um script real, você poderia levantar um erro aqui para parar a execução:
    # raise ValueError("GOOGLE_API_KEY não encontrada. Interrompendo.")

Chave API configurada com sucesso!


## 04. Importing Models

This code block demonstrates the essential steps to interact with the Gemini API
and generate content from a specific model.

In [ ]:
# CELL 5: Listing all Google Gemini Models available

"""
The general flow is:
1.  (Optional, but recommended for exploration): List available models.
    This can be done with a loop like:
    for available_model in client.models.list():
        print(available_model.name)
    This step helps identify the correct names of the models you can use.
"""

print("Modelos disponíveis que suportam 'generateContent':")

for m in genai.list_models():
  # Você pode querer filtrar modelos que podem gerar conteúdo, por exemplo
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

Modelos disponíveis que suportam 'generateContent':
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models

In [ ]:
# CELL 6: Define the model name for the API call.

# Escolhemos um modelo da API Gemini.
# Modelos como "gemini-1.5-flash-latest" ou "gemini-1.5-pro-latest" são boas escolhas
# por suportarem multimodalidade (processamento de arquivos como PDF) e seguirem bem as instruções.
# O modelo "gemini-2.0-flash" usado originalmente pode ser uma versão experimental ou mais antiga.
# Recomenda-se verificar a documentação oficial para os modelos mais recentes e suas capacidades.

model_name = "models/gemini-2.0-flash" # Atualizado para um modelo recomendado

print(f"Modelo selecionado: {model_name}")

Modelo selecionado: models/gemini-2.0-flash


## 05. ChatBot Create

Definição do "Cérebro" do Chatbot: Instrução do Sistema e Configurações

Esta é uma das partes mais importantes. A `system_instruction` é um prompt detalhado que guia o comportamento do modelo Gemini. Definimos aqui como ele deve interagir, processar documentos, lidar com cálculos, e quais são suas limitações.

Também configuramos:
*   `generation_config`: Parâmetros como `temperature` (criatividade vs. factualidade).
*   `safety_settings`: Para filtrar conteúdo indesejado.
*   `model_instance`: Criamos a instância do modelo com todas essas configurações.
*   `chat`: Iniciamos uma sessão de chat com o modelo.

In [ ]:
# CELL 7: System Instruction, Generation Config, Safety Settings, and Model Initialization

system_instruction_diabetes = """
Você é um assistente pessoal especializado em diabetes, projetado para ser conciso, útil e, acima de tudo, seguro.
Seu objetivo é auxiliar usuários a entenderem melhor sua condição e tratamento, com base nas informações que eles fornecem, incluindo o conteúdo de documentos médicos (como receitas em PDF) que eles podem fazer upload.

REGRAS DE INTERAÇÃO:

1.  PROCESSAMENTO DE DOCUMENTOS (PDFs, etc.):
    *   Se o usuário fizer upload de um documento (ex: receita médica em PDF), VOCÊ DEVE analisar o conteúdo desse documento para responder às perguntas relacionadas a ele.
    *   Extraia informações relevantes como: nomes de medicamentos, insumos médicos, equipamentos, suas dosagens/apresentações, frequência de uso, quantidades prescritas (mensais ou totais), instruções específicas, relação insulina/carboidrato, fator de sensibilidade, meta glicêmica, etc.
    *   Ao responder, mencione que a informação foi extraída do documento fornecido para dar contexto.
    *   Se o documento não contiver a informação solicitada, informe isso claramente.
    *   Se você não conseguir processar ou entender uma parte do documento, informe o usuário.

2.  EXTRAÇÃO PARA CONSULTA DE PREÇOS PELO SISTEMA EXTERNO:
    *   Após analisar uma receita, se o sistema Python solicitar, VOCÊ DEVE listar todos os medicamentos, insumos médicos e equipamentos mencionados, incluindo suas apresentações/dosagens e as quantidades prescritas (ex: mensais, por ciclo, ou compra única).
    *   **Forneça esta lista em formato JSON**, se solicitado pelo sistema. Cada item da lista deve ser um objeto com, no mínimo, as chaves: "item" (string, nome do produto), "apresentacao" (string, ex: "100 UI/mL", "4mm", ou null se não aplicável), e "quantidade_descrita" (string, ex: "5 canetas", "200 agulhas mensais", "1 aparelho"). Se possível, adicione uma chave "tipo" (string: "medicamento", "insumo", "equipamento").
    *   Esta lista será usada pelo sistema Python para tentar buscar uma estimativa de preços em sites de farmácias. Você (o LLM) não fará a busca de preços diretamente na web.

3.  DOSAGEM DE INSULINA E CÁLCULOS:
    *   Se o usuário perguntar sobre cálculo de dose de insulina e tiver fornecido um documento com os parâmetros (Relação Insulina/Carboidrato, Fator de Sensibilidade, Meta Glicêmica),
        utilize ESSES DADOS do documento para o cálculo.
    *   Se o documento não contiver todos os parâmetros necessários, ou se o usuário não fornecer um documento, mas fornecer os parâmetros NUMÉRICOS diretamente, você PODE realizar um
        cálculo auxiliar baseado EXCLUSIVAMENTE nos números fornecidos.
    *   Pergunte pela glicemia atual SOMENTE se o cálculo for para correção de hiperglicemia e essa informação não estiver no documento ou não for fornecida.
    *   SEMPRE após qualquer cálculo de dose, exiba o seguinte AVISO EM NEGRITO E COM DESTAQUE:
        '**ATENÇÃO: Este é um cálculo auxiliar baseado ESTRITAMENTE nas informações fornecidas (seja do documento ou diretamente por você). Ele serve APENAS como uma referência e
        NÃO substitui, de forma alguma, o aconselhamento médico profissional, a consulta com seu médico ou educador em diabetes, nem sua própria responsabilidade e julgamento.
        SEMPRE confirme todos os cálculos e a dosagem final com seu profissional de saúde antes de administrar qualquer insulina. A decisão final e a responsabilidade pela
        dosagem são SUAS e do seu profissional de saúde.**'

4.  SEGURANÇA E LIMITAÇÕES:
    *   Se o usuário não fornecer um documento nem os parâmetros numéricos para cálculo de dose, ou se a pergunta for muito genérica sobre dosagem sem dados concretos,
        REFORCE que você não pode fornecer conselhos médicos específicos sobre dosagem sem esses dados e que ele DEVE consultar um profissional de saúde.
    *   NÃO FAÇA DIAGNÓSTICOS. NÃO SUGIRA ALTERAÇÕES DE TRATAMENTO.
    *   Para todas as outras perguntas (o que é diabetes, tipos de alimentos, etc.), responda de forma informativa e útil, mas evite qualquer coisa que possa ser interpretada
        como conselho médico específico não embasado nos dados fornecidos pelo usuário/documento.

5.  TOM E CONCISÃO:
    *   Mantenha um tom empático, mas firme quanto às questões de segurança.
    *   Seja o mais conciso possível em suas respostas, mas sem perder a clareza e a utilidade.
"""

# Configuração de como o modelo deve gerar as respostas.
generation_config = genai.types.GenerationConfig(
    temperature=0.2 # Baixa temperatura para extração factual e JSON
)

# Define quais tipos de conteúdo prejudicial devem ser bloqueados e com qual rigor.
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

# Cria a instância do modelo generativo com todas as configurações definidas.
model_instance = genai.GenerativeModel(
    model_name=model_name,
    generation_config=generation_config,
    system_instruction=system_instruction_diabetes,
    safety_settings=safety_settings
)

# Inicia uma sessão de chat com o modelo. O histórico da conversa será mantido neste objeto.
chat = model_instance.start_chat(history=[])

print(f"Chatbot Auxiliar de Diabetes (Modelo: {model_name}) pronto!")
print("Digite sua pergunta ou 'fim' para sair.")
print("Se quiser que eu analise uma receita, digite 'upload' e siga as instruções.")
print("-" * 70)

Chatbot Auxiliar de Diabetes (Modelo: models/gemini-2.0-flash) pronto!
Digite sua pergunta ou 'fim' para sair.
Se quiser que eu analise uma receita, digite 'upload' e siga as instruções.
----------------------------------------------------------------------


### Funções Auxiliares: Extração de Quantidade e Simulação de Busca de Preços

Antes do loop principal do chat, definimos algumas funções Python que nos ajudarão:

1.  **`extrair_info_quantidade`**: Tenta extrair de uma string (como "2 caixas com 50 unidades" ou "5 canetas") o número principal de embalagens e, se aplicável, quantas unidades básicas (comprimidos, ml, agulhas) existem em cada embalagem. Isso é crucial para calcular o custo total corretamente.
2.  **`simular_preco_unitario_embalagem_farmacia_v2`**: **Esta função SIMULA a busca do preço da embalagem padrão de um item em uma farmácia específica.** Em uma aplicação real, esta função conteria a lógica complexa de web scraping (usando `requests` e `BeautifulSoup`) ou chamadas a APIs de farmácias. Para este protótipo, ela retorna preços e informações de embalagem pré-definidos (hardcoded) com uma pequena variação para simular diferentes farmácias.
3.  **`obter_precos_simulados_farmacias_v3`**: Orquestra a chamada à função de simulação para as três redes de farmácias alvo (Drogasil, Droga Raia, Drogaria São Paulo).

**Importante sobre a Simulação de Preços:**

*   Os preços retornados são **FICTÍCIOS** e servem apenas para demonstrar o fluxo do chatbot.
*   A lógica de correspondência de produtos e a estrutura de preços são simplificadas.
*   Uma implementação real exigiria um esforço significativo para desenvolver e manter os web scrapers ou integrações com APIs.

In [ ]:
# CELL 8: Funções Auxiliares e de Scraping (Simuladas) - COM LÓGICA DE CÁLCULO AJUSTADA

# Variável global para armazenar os itens extraídos da receita atual
itens_da_receita_atual = None

def extrair_info_quantidade(texto_quantidade_descrita, nome_item_para_contexto=""):
    """
    Extrai o número principal de unidades/embalagens da receita E a quantidade de unidades
    por embalagem, se especificado (ex: "caixa com 50").
    Retorna (numero_embalagens_receita, unidades_por_embalagem_na_receita_se_aplicavel)
    """
    if not texto_quantidade_descrita:
        return 1, None # 1 embalagem, unidades por embalagem não especificadas

    texto_lower = str(texto_quantidade_descrita).lower()

    numero_embalagens = 1
    unidades_por_embalagem_receita = None

    # Tenta encontrar "X caixas/embalagens com Y unidades"
    # Ex: "2 caixas com 50 unidades", "3 refis de 3ml" (aqui 3ml é detalhe da apresentação)
    match_complexo = re.search(r'(\d+)\s*(caixas?|cx|embalagens?|frascos?|bisnagas?|potes?|sachês?)\s*(com|de|contendo)\s*(\d+)\s*(unidades?|un|comp|cápsulas|ml|g|mg|mcg)', texto_lower)
    if match_complexo:
        numero_embalagens = int(match_complexo.group(1))
        unidades_por_embalagem_receita = int(match_complexo.group(4))
    else:
        # Se não for complexo, pega o primeiro número como número de embalagens/itens principais
        match_simples = re.search(r'(\d+)', texto_lower)
        if match_simples:
            numero_embalagens = int(match_simples.group(0))
            # Tenta ver se há uma quantidade unitária após (ex: "agulhas de 4mm")
            # Esta parte é mais difícil de generalizar sem contexto do DB de preços.
            # Por ora, focamos no número principal de embalagens.

    return numero_embalagens, unidades_por_embalagem_receita


# --- FUNÇÕES DE SCRAPING (SIMULADAS) ---
# A simulação agora focará em retornar o PREÇO UNITÁRIO DA EMBALAGEM DE VENDA
# e, crucialmente, QUANTAS UNIDADES BÁSICAS (comprimidos, ml, agulhas) ESSA EMBALAGEM CONTÉM.
def simular_preco_unitario_embalagem_farmacia_v2(nome_farmacia, nome_produto_query):
    """
    Simula o PREÇO UNITÁRIO DA EMBALAGEM de venda em uma farmácia e
    QUANTAS UNIDADES BÁSICAS (ex: comprimidos, ml, agulhas) essa embalagem contém.
    """
    #print(f"DEBUG: [SIMULAÇÃO] Buscando preço unit. embalagem '{nome_produto_query}' na {nome_farmacia}...")
    time.sleep(0.1) # Simula delay

    preco_unit_embalagem = None
    unidades_na_embalagem_db = 1 # Default: embalagem contém 1 unidade básica (ex: 1 caneta, 1 sensor)
    unidade_basica_db = "unidade" # Default: "unidade"

    nome_produto_query_lower = nome_produto_query.lower().strip()
    nome_base_produto = re.sub(r'\b\d+\s*(canetas?|refis|agulhas|sensores|caixas|comprimidos|unidades|ml|mg|g)\b', '', nome_produto_query_lower, flags=re.IGNORECASE).strip()
    nome_base_produto = re.sub(r'\s\s+', ' ', nome_base_produto)

    #print(f"DEBUG: [SIMULAÇÃO] Nome base para busca de preço unitário: '{nome_base_produto}'")

    # SIMULAÇÃO DE PREÇOS E UNIDADES POR EMBALAGEM NO "BANCO DE DADOS" DA FARMÁCIA
    if "insulina degludeca" in nome_base_produto or "tresiba" in nome_base_produto:
        preco_unit_embalagem = round(140.00 + hash(nome_farmacia + nome_base_produto) % 20, 2)
        unidades_na_embalagem_db = 1; unidade_basica_db = "caneta"
    elif "insulina fiasp" in nome_base_produto:
        preco_unit_embalagem = round(65.00 + hash(nome_farmacia + nome_base_produto) % 15, 2)
        unidades_na_embalagem_db = 1; unidade_basica_db = "refil/caneta"
    elif "agulhas para caneta de insulina" in nome_base_produto or "agulha" in nome_base_produto:
        preco_unit_embalagem = round(25.00 + hash(nome_farmacia + nome_base_produto) % 5, 2) # Preço da CAIXA
        unidades_na_embalagem_db = 100; unidade_basica_db = "agulha" # Caixa com 100 agulhas
    elif "sensor de glicose intersticial freestyle libre" in nome_base_produto:
        preco_unit_embalagem = round(280.00 + hash(nome_farmacia + nome_base_produto) % 30, 2)
        unidades_na_embalagem_db = 1; unidade_basica_db = "sensor"
    elif "tiras reagentes para freestyle optium neo abbot" in nome_base_produto:
        preco_unit_embalagem = round(70.00 + hash(nome_farmacia + nome_base_produto) % 10, 2) # Preço da CAIXA
        unidades_na_embalagem_db = 50; unidade_basica_db = "tira" # Caixa com 50 tiras
    elif "monitor freestyle libre" in nome_base_produto:
        preco_unit_embalagem = round(250.00 + hash(nome_farmacia + nome_base_produto) % 20, 2)
        unidades_na_embalagem_db = 1; unidade_basica_db = "aparelho"
    elif "losartana potassica 50mg" in nome_base_produto or "losartana 50mg" in nome_base_produto: # Exemplo cardio
        preco_unit_embalagem = round(15.00 + hash(nome_farmacia + nome_base_produto) % 5, 2) # Preço da CAIXA
        unidades_na_embalagem_db = 30; unidade_basica_db = "comprimido" # Caixa com 30 comprimidos
    elif "sinvastatina 20mg" in nome_base_produto: # Exemplo cardio
        preco_unit_embalagem = round(20.00 + hash(nome_farmacia + nome_base_produto) % 7, 2) # Preço da CAIXA
        unidades_na_embalagem_db = 30; unidade_basica_db = "comprimido" # Caixa com 30 comprimidos

    if preco_unit_embalagem is not None:
        link_simulado_query = nome_base_produto.replace(' ', '+') if nome_base_produto else nome_produto_query.replace(' ', '+')
        return {
            "farmacia": nome_farmacia,
            "produto_buscado_sim": nome_base_produto if nome_base_produto else nome_produto_query,
            "preco_embalagem_sim": preco_unit_embalagem,
            "unidades_por_embalagem_sim": unidades_na_embalagem_db,
            "unidade_basica_sim": unidade_basica_db,
            "link_simulado": f"https://www.{nome_farmacia.lower().replace(' ', '')}.com.br/search?w={link_simulado_query}"
        }
    return None

def obter_precos_simulados_farmacias_v3(item_info_extraida):
    """
    Chama as funções de scraping (simuladas) para as farmácias e retorna
    uma lista de dicts com "preco_embalagem_sim" e "unidades_por_embalagem_sim".
    """
    nome_item = item_info_extraida.get("item")
    apresentacao = item_info_extraida.get("apresentacao") # Pode ser usado para refinar a query

    if not nome_item:
        return []

    # A query para simulação de preço deve ser o mais próximo possível do que o usuário veria na farmácia
    # para a embalagem padrão.
    query_para_preco_embalagem = f"{nome_item} {apresentacao if apresentacao else ''}".strip()

    resultados_farmacias = []
    farmacias_para_buscar = ["Drogasil", "Droga Raia", "Drogaria Sao Paulo"]

    for nome_farmacia in farmacias_para_buscar:
        try:
            resultado_sim = simular_preco_unitario_embalagem_farmacia_v2(nome_farmacia, query_para_preco_embalagem)
            if resultado_sim:
                resultados_farmacias.append(resultado_sim)
        except Exception as e_scrape_sim:
            print(f"DEBUG: Erro ao simular scraping com {nome_farmacia} para '{query_para_preco_embalagem}': {e_scrape_sim}")

    return resultados_farmacias

### Loop Interativo Principal do Chatbot

A célula abaixo contém o loop `while True` que mantém o chatbot em execução, aguardando a entrada do usuário. Ela lida com:

1.  **Entrada do Usuário:** Captura o que o usuário digita.
2.  **Comando "fim":** Encerra o chat e tenta limpar arquivos no servidor Gemini.
3.  **Comando "upload":**
    *   Permite ao usuário enviar um arquivo PDF (receita).
    *   O arquivo é salvo temporariamente no Colab.
    *   É feito o upload do arquivo para a API Gemini.
    *   **Extração de Itens:** Um prompt específico é enviado ao Gemini para analisar o PDF e retornar uma lista de medicamentos/insumos em formato JSON. Essa lista é armazenada para uso posterior na estimativa de preços.
    *   O arquivo temporário no Colab é removido.
4.  **Estimativa de Preços:**
    *   Se o usuário pergunta sobre "preço", "custo", etc., e uma lista de itens foi extraída anteriormente:
        *   Para cada item da lista, a função `obter_precos_simulados_farmacias_v3` é chamada.
        *   Esta função (atualmente simulada) "consulta" as três farmácias.
        *   A lógica calcula o número de embalagens necessárias com base na quantidade da receita e no conteúdo das embalagens (simulado).
        *   O custo total para o item é estimado (preço da embalagem * número de embalagens).
        *   Um resumo dos custos por item e um total geral para a receita são apresentados, com **avisos importantes sobre a natureza simulada dos preços**.
5.  **Perguntas Gerais:**
    *   Para outras perguntas, o prompt do usuário (e o arquivo PDF, se carregado) é enviado ao Gemini para uma resposta conversacional.

**Gerenciamento de Arquivos:** O chatbot tenta deletar arquivos do servidor Gemini quando um novo é carregado ou quando o chat é encerrado, para gerenciar os recursos.

In [ ]:
# CELL 9: Interactive Chat Loop with PDF processing & Corrected Price Calculation

# Globais, se não definidos na célula anterior, defina aqui:
if 'uploaded_file_gemini' not in globals():
    uploaded_file_gemini = None
if 'current_colab_temp_filename' not in globals():
    current_colab_temp_filename = None
if 'itens_da_receita_atual' not in globals():
    itens_da_receita_atual = None


while True:
    prompt_usuario = input("Você: ").strip()
    if not prompt_usuario:
        continue

    if prompt_usuario.lower() == "fim":
        print("Chatbot: Até logo! Lembre-se de sempre consultar seu médico para questões de saúde.")
        if uploaded_file_gemini:
            try:
                print(f"Chatbot: Tentando remover o arquivo '{uploaded_file_gemini.display_name}' (ID: {uploaded_file_gemini.name}) do servidor Gemini...")
                genai.delete_file(uploaded_file_gemini.name)
                print(f"Chatbot: Arquivo '{uploaded_file_gemini.display_name}' removido com sucesso.")
            except Exception as e_del:
                print(f"Chatbot: Aviso - não foi possível remover o arquivo '{uploaded_file_gemini.display_name}': {e_del}")
        break

    # Lógica para upload de arquivo
    if "upload" in prompt_usuario.lower() or "carregar arquivo" in prompt_usuario.lower():
        itens_da_receita_atual = None # Resetar ao fazer novo upload
        # ... (restante da lógica de upload e extração JSON como na sua última versão fornecida) ...
        # ... Certifique-se que a extração JSON popule `itens_da_receita_atual` ...
        print("Chatbot: Por favor, selecione o arquivo (PDF, TXT, etc.) da sua receita para upload.")
        try:
            uploaded_colab_files = files.upload()
            if not uploaded_colab_files:
                print("Chatbot: Nenhum arquivo foi selecionado.")
                continue

            colab_filename = next(iter(uploaded_colab_files))
            file_bytes = uploaded_colab_files[colab_filename]
            current_colab_temp_filename = colab_filename

            with open(current_colab_temp_filename, 'wb') as f_temp:
                f_temp.write(file_bytes)

            print(f"Chatbot: Arquivo '{colab_filename}' recebido. Enviando para análise do Gemini...")
            if uploaded_file_gemini:
                try: genai.delete_file(uploaded_file_gemini.name)
                except: pass
                uploaded_file_gemini = None

            uploaded_file_gemini = genai.upload_file(path=current_colab_temp_filename, display_name=colab_filename)
            print(f"Chatbot: Arquivo '{uploaded_file_gemini.display_name}' enviado ao Gemini.")

            try:
                os.remove(current_colab_temp_filename)
                current_colab_temp_filename = None
            except: pass

            if uploaded_file_gemini:
                print("Chatbot: Analisando a receita para extrair a lista de medicamentos e insumos...")
                try:
                    prompt_extracao_json = (
                        "Com base no documento que acabei de fornecer, por favor, identifique e liste todos os medicamentos, "
                        "insumos médicos e equipamentos prescritos. Para cada item, forneça: "
                        "1. O nome do item (chave 'item'). "
                        "2. A apresentação ou dosagem, se aplicável (chave 'apresentacao'). "
                        "3. A quantidade descrita na receita (ex: '3 canetas mensais', '1 caixa com 100 agulhas', '1 aparelho') (chave 'quantidade_descrita'). "
                        "4. O tipo de item (ex: 'medicamento', 'insumo', 'equipamento') (chave 'tipo'). "
                        "Formate sua resposta como uma lista de objetos JSON válidos. Certifique-se de que o JSON seja válido e completo."
                    )
                    resposta_extracao = chat.send_message([prompt_extracao_json, uploaded_file_gemini])

                    texto_resposta_extracao = resposta_extracao.text
                    #print(f"DEBUG: Resposta crua da extração JSON: {texto_resposta_extracao[:300]}...")

                    match_json = re.search(r'```json\s*([\s\S]*?)\s*```', texto_resposta_extracao)
                    json_str_para_parsear = None
                    if match_json:
                        json_str_para_parsear = match_json.group(1)
                    elif texto_resposta_extracao.strip().startswith("[") and texto_resposta_extracao.strip().endswith("]"):
                        json_str_para_parsear = texto_resposta_extracao.strip()

                    if json_str_para_parsear:
                        try:
                            itens_da_receita_atual = json.loads(json_str_para_parsear)
                            if isinstance(itens_da_receita_atual, list) and \
                               all(isinstance(it, dict) and "item" in it and "quantidade_descrita" in it for it in itens_da_receita_atual):
                                print(f"Chatbot: Lista de {len(itens_da_receita_atual)} itens extraída com sucesso!")
                            else:
                                print("Chatbot: A extração JSON não retornou o formato esperado.")
                                itens_da_receita_atual = None
                        except json.JSONDecodeError as e_json:
                            print(f"Chatbot: Erro ao decodificar o JSON da lista de itens: {e_json}")
                            itens_da_receita_atual = None
                    else:
                        print("Chatbot: Não foi possível extrair um bloco JSON da resposta do LLM para a lista de itens.")
                        itens_da_receita_atual = None
                except Exception as e_ext:
                    print(f"Chatbot: Erro durante a extração da lista de itens: {e_ext}")
                    itens_da_receita_atual = None

            print("Chatbot: Arquivo processado. Faça sua pergunta ou peça uma estimativa de custos.")
            continue

        except Exception as e_upload_geral:
            print(f"Chatbot: Erro geral no processo de upload: {e_upload_geral}")
            if current_colab_temp_filename and os.path.exists(current_colab_temp_filename):
                try: os.remove(current_colab_temp_filename)
                except: pass
            uploaded_file_gemini = None
            continue

    # --- Processamento de perguntas do usuário (incluindo estimativa de preço) ---
    if prompt_usuario:
        # Lógica para estimativa de preço
        if ("preço" in prompt_usuario.lower() or "custo" in prompt_usuario.lower() or "valor" in prompt_usuario.lower() or "orçamento" in prompt_usuario.lower()) and \
           ("medicamentos" in prompt_usuario.lower() or "receita" in prompt_usuario.lower() or "insumos" in prompt_usuario.lower() or "total" in prompt_usuario.lower() or "itens" in prompt_usuario.lower()):

            if not uploaded_file_gemini:
                print("Chatbot: Por favor, primeiro faça o upload da sua receita (comando 'upload').")
                continue
            if not itens_da_receita_atual or not isinstance(itens_da_receita_atual, list):
                print("Chatbot: Não consegui obter a lista de itens da receita. Tente o upload novamente ou verifique se a receita está legível.")
                continue

            print("\nChatbot: Buscando estimativas de preços nas farmácias Drogasil, Droga Raia e Drogaria São Paulo (usando DADOS SIMULADOS)...")

            resumo_custos_formatado = []
            custo_geral_total_receita_min = 0.0
            custo_geral_total_receita_max = 0.0
            pelo_menos_um_item_com_preco = False

            for item_info_llm in itens_da_receita_atual:
                nome_item_display = item_info_llm.get("item", "Item Desconhecido")
                qtd_descrita_pelo_llm = item_info_llm.get("quantidade_descrita", "1")

                # Extrai a quantidade de embalagens/itens principais da receita e, se houver, unidades por embalagem na receita
                qtd_embalagens_receita, _ = extrair_info_quantidade(qtd_descrita_pelo_llm, nome_item_display)

                print(f"\n  Buscando preços para: {nome_item_display} (Receita pede: {qtd_descrita_pelo_llm} -> {qtd_embalagens_receita} embalagem(ns) principal(is))...")

                # Retorna [{farmacia, produto_buscado_sim, preco_embalagem_sim, unidades_por_embalagem_sim, unidade_basica_sim, link_simulado}, ...]
                resultados_simulados_item = obter_precos_simulados_farmacias_v3(item_info_llm)

                if resultados_simulados_item:
                    precos_embalagem_encontrados_para_item = [res['preco_embalagem_sim'] for res in resultados_simulados_item if res.get('preco_embalagem_sim') is not None]

                    if not precos_embalagem_encontrados_para_item:
                        resumo_custos_formatado.append(f"  - {nome_item_display} ({qtd_descrita_pelo_llm}): Não foram encontrados preços (simulados) de embalagem.")
                        continue

                    preco_embalagem_min_sim = min(precos_embalagem_encontrados_para_item)
                    preco_embalagem_max_sim = max(precos_embalagem_encontrados_para_item)

                    # Assume que a primeira simulação tem a info correta de unidades por embalagem do DB
                    unidades_por_embalagem_no_db = resultados_simulados_item[0].get('unidades_por_embalagem_sim', 1)
                    unidade_basica_no_db = resultados_simulados_item[0].get('unidade_basica_sim', 'unidade')


                    # LÓGICA DE CÁLCULO CORRIGIDA
                    # Se a receita pede uma quantidade total de unidades básicas (ex: 200 agulhas)
                    # E o preço simulado é por embalagem que contém X unidades básicas (ex: caixa com 100 agulhas)
                    # Precisamos calcular quantas embalagens são necessárias.

                    numero_de_embalagens_a_comprar = qtd_embalagens_receita # Default

                    # Heurística: Se a "quantidade_descrita" na receita parecer ser unidades básicas
                    # e for maior que o conteúdo de uma embalagem do DB, recalculamos o número de embalagens.
                    # Ex: Receita "200 agulhas", DB "preço por caixa de 100". Comprar 2 caixas.
                    # Ex: Receita "5 canetas", DB "preço por caneta". Comprar 5 canetas. (aqui qtd_embalagens_receita já é 5)
                    if (unidade_basica_no_db in qtd_descrita_pelo_llm.lower() or "unidades" in qtd_descrita_pelo_llm.lower()) and \
                       qtd_embalagens_receita > unidades_por_embalagem_no_db and \
                       not ("caixa" in qtd_descrita_pelo_llm.lower() or "embalagem" in qtd_descrita_pelo_llm.lower()): # Evitar se a receita já especifica caixas
                        numero_de_embalagens_a_comprar = math.ceil(qtd_embalagens_receita / unidades_por_embalagem_no_db)
                        print(f"    Ajuste: Receita pede {qtd_embalagens_receita} {unidade_basica_no_db}(s), "
                              f"cada embalagem simulada tem {unidades_por_embalagem_no_db}. "
                              f"Serão necessárias {numero_de_embalagens_a_comprar} embalagens.")
                    else:
                         # Caso contrário, assumimos que qtd_embalagens_receita é o número de embalagens a comprar.
                         # Ex: "5 canetas" (onde cada caneta é uma embalagem)
                         # Ex: "2 caixas" (onde cada caixa é uma embalagem)
                         numero_de_embalagens_a_comprar = qtd_embalagens_receita


                    custo_total_item_min_calculado = preco_embalagem_min_sim * numero_de_embalagens_a_comprar
                    custo_total_item_max_calculado = preco_embalagem_max_sim * numero_de_embalagens_a_comprar

                    custo_geral_total_receita_min += custo_total_item_min_calculado
                    custo_geral_total_receita_max += custo_total_item_max_calculado
                    pelo_menos_um_item_com_preco = True

                    detalhe_item_str = f"  - {nome_item_display} (Prescrito: {qtd_descrita_pelo_llm} -> {numero_de_embalagens_a_comprar} emb. de ~{unidades_por_embalagem_no_db} {unidade_basica_no_db}): "
                    if custo_total_item_min_calculado == custo_total_item_max_calculado:
                        detalhe_item_str += f"Custo total estimado R$ {custo_total_item_min_calculado:.2f} "
                    else:
                        detalhe_item_str += f"Custo total estimado entre R$ {custo_total_item_min_calculado:.2f} e R$ {custo_total_item_max_calculado:.2f} "

                    farmacias_consultadas_str = ", ".join(sorted(list(set(res['farmacia'] for res in resultados_simulados_item))))
                    detalhe_item_str += f"(Preço por embalagem: R${preco_embalagem_min_sim:.2f} a R${preco_embalagem_max_sim:.2f} em {farmacias_consultadas_str})"
                    resumo_custos_formatado.append(detalhe_item_str)
                else:
                    resumo_custos_formatado.append(f"  - {nome_item_display} ({qtd_descrita_pelo_llm}): Não foram encontrados preços (simulados).")

            if resumo_custos_formatado:
                print("\n--- ESTIMATIVA DE CUSTOS (SIMULADA / FARMÁCIAS ESPECÍFICAS) ---")
                for detalhe_str in resumo_custos_formatado:
                    print(detalhe_str)

                if pelo_menos_um_item_com_preco:
                    if custo_geral_total_receita_min == custo_geral_total_receita_max:
                         print(f"\n  **CUSTO TOTAL GERAL DA RECEITA (SIMULADO): R$ {custo_geral_total_receita_min:.2f}**")
                    else:
                         print(f"\n  **CUSTO TOTAL GERAL DA RECEITA (SIMULADO): Entre R$ {custo_geral_total_receita_min:.2f} e R$ {custo_geral_total_receita_max:.2f}**")
                else:
                    print("\n  Não foi possível calcular um custo total geral para a receita com os dados simulados.")

            print(("\n**ATENÇÃO MUITO IMPORTANTE:** Estes valores são **ESTIMATIVAS e SIMULADAS** para Drogasil, Droga Raia e Drogaria São Paulo. "
                   "Eles **NÃO REFLETEM** os preços reais na sua farmácia HOJE, com seu plano de saúde, ou descontos específicos. "
                   "Os preços de medicamentos e insumos variam enormemente e mudam com frequência. "
                   "**Consulte SEMPRE os sites ou lojas físicas dessas farmácias para obter os valores exatos.**\n"))
            continue

        # --- Envio normal de mensagem para o LLM para outras perguntas ---
        try:
            message_parts_para_llm = [prompt_usuario]
            if uploaded_file_gemini:
                 message_parts_para_llm.append(uploaded_file_gemini)

            print(f"Chatbot (Contexto: {'Arquivo ' + uploaded_file_gemini.display_name if uploaded_file_gemini else 'Sem arquivo'}): Processando sua pergunta...")
            response = chat.send_message(message_parts_para_llm)
            print(f"Chatbot: {response.text}\n")

        except exceptions.GoogleAPIError as e_api_msg:
            print(f"Chatbot: ERRO DE API DO GOOGLE: {e_api_msg}")
        except Exception as e_msg:
            print(f"Chatbot: Desculpe, erro ao processar sua mensagem: {e_msg}")

print("Chat encerrado.")

Chatbot (Contexto: Arquivo FILIPE SILVEIRA GONCALVES 2025.pdf): Processando sua pergunta...


Chatbot: ERRO DE API DO GOOGLE: 403 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You do not have permission to access the File pca8vz41nwey or it may not exist.
